# Nuts and Bolts of `e3nn`

## tutorial by: Tess E. Smidt

## Everything in `e3nn` starts with a "representation list"
This defines the data types of your spherical tensor. At every step in the network, there needs to be an `Rs` that identifies the entries in your spherical tensor.

Let's start with a simple example where we are starting off with a benezene molecule

In [ ]:
import torch
import e3nn
torch.set_default_dtype(torch.float64)

In [ ]:
Rs = [(2, 0)] # Two (2) scalar (L=0) channels: hydrogen and carbon

# 3D coordinates of the atoms of the molecule
C_geo = torch.tensor(
    [[ 0.     ,  1.40272,  0.     ],
     [-1.21479,  0.70136,  0.     ],
     [-1.21479, -0.70136,  0.     ],
     [ 0.     , -1.40272,  0.     ],
     [ 1.21479, -0.70136,  0.     ],
     [ 1.21479,  0.70136,  0.     ]]
)
H_geo = torch.tensor(
    [[ 0.     ,  2.49029,  0.     ],
     [-2.15666,  1.24515,  0.     ],
     [-2.15666, -1.24515,  0.     ],
     [ 0.     , -2.49029,  0.     ],
     [ 2.15666, -1.24515,  0.     ],
     [ 2.15666,  1.24515,  0.     ]]
)
geometry = torch.cat([C_geo, H_geo], axis=-2)

# and features on each atom
C_input = torch.tensor([[0., 1.] for i in range(C_geo.shape[-2])])
H_input = torch.tensor([[1., 0.] for i in range(H_geo.shape[-2])])
input = torch.cat([C_input, H_input])

## Building a convolution
A traditional convolution applies a filter function $W(\vec{r_ij})$ to an input $I_j$ with an element-wise multiply $\odot$ and sums for all $j$ neighboring $i$.

$\text{Traditional Convolution} = \sum_{j \in n_i} W(\vec{r}) \odot I_j$

Our convolution differs in two ways:
* $W(\vec{r})$  is constrained to be seperable into a learned radial function $R(r)$ and spherical harmonics $Y_{lm}(\hat{r})$
* Our datatypes are geometric tensors so we replace the element-wise multiply with a tensor product and Clebsch-Gordon coefficients

$\text{E(3) Convolution} = \sum_{j \in n_{i}} R(r) Y_{lm}(\hat{r}) \otimes I_j$

We will use 3 classes to build our convolutions an instance of a class based on `e3nn.radial.RadialModel`, a instance of `e3nn.kernel.Kernel` and an instance of `e3nn.point.operations.Convolution`.

# RadialModel
We are going to use the `CosineBasisModel` function for our basis functions
They look a bit like Gaussians, but they don't have long tails. The function is equal to $\text{cos}^2(arg)$ modulo normalization constants. You can use any radial basis you like, just make sure it's well normalized.

In [ ]:
# Inside e3nn.radial.CosineBasisModel
import math
import matplotlib.pyplot as plt
%matplotlib inline

max_radius = 3.0
number_of_basis = 3
radii = torch.linspace(0, max_radius, steps=number_of_basis)
step = radii[1] - radii[0]
basis = lambda x: x.div(step).add(1).relu().sub(2).neg().relu().add(1).mul(math.pi / 2).cos().pow(2)

x = torch.linspace(-max_radius, max_radius, 1000)
plt.plot(x, basis(x))

## Using functools.partial to set up classes
[Mario Geiger](https://mariogeiger.ch/) is the [BDFL](https://en.wikipedia.org/wiki/Benevolent_dictator_for_life) of `e3nn`. `functools.partial` is Mario's favorite python function so we'll use it to define our radial function, kernel, convolution, and layers.

We are going to define `RadialModel` by specifying every single argument of `CosineBasisModel` using `functools.partial` EXCEPT out_dim which will be passed later. The `CosineBasisModel` will then have a `radial_layer=2` layer fully connected neural network with hidden dimention `h=100` applied to the basis functions vectors.

$$R_{c}(r_{ij}) = W_{ch} \sigma(W_{hb} B_b(r_{ij}))$$

where $b$ is the number of basis funcitons and $c$ is unintuitively the number of "paths" -- the number of valid combinations of $L_{input}$, $L_{filter}$, and $L_{output}$.

In [ ]:
# e3nn has operations for point sets and for 3D images. We will be using points.
import e3nn.point
import e3nn.radial
from functools import partial
from e3nn.non_linearities import rescaled_act

# We are going to define RadialModel by specifying every single argument
# of CosineBasisModel EXCEPT out_dim which will be passed later
radial_layers = 2
sp = rescaled_act.Softplus(beta=5)
RadialModel = partial(e3nn.radial.CosineBasisModel, max_radius=max_radius,
                      number_of_basis=number_of_basis, h=100,
                      L=radial_layers, act=sp)

## Setting up the kernel
Now we define the kernel or filter function by passing `RadialModel` in yet another call to `partial`. This time we are specifying all arguements to `Kernel` EXCEPT `Rs_in` and `Rs_out`, which again will be passed later.

One of the arguements to `Kernel` is `sh` which specifies which spherical harmonics to use. In the vast majority of cases, this should be set to the default `o3.spherical_harmonics_xyz`.

### ...but just in case...
If you need to backpropogate to your input geometry, then you must use `e3nn.o3.spherical_harmonics_xyz_backwardable`. This should only be used for low $L$ as (for the current implementation) gradients become unstable for high $L$.

In [ ]:
import e3nn.kernel
K = partial(e3nn.kernel.Kernel, RadialModel=RadialModel)

## And finally we define our convolution

In [ ]:
import e3nn.point.operations

# If we wish to pass the convolution to a layer definition
C = partial(e3nn.point.operations.Convolution, K)

# Or alternatively, if we want to use the convolution directly,
# we need to specify the `Rs` of the input and output
Rs_in = [(2, 0)]
Rs_out = [(4, 0), (4, 1), (4, 2)]
convolution = e3nn.point.operations.Convolution(K(Rs_in, Rs_out))

## Nonlinearities
To preserve equivariance, we need to use equivariant nonlinearities. One such example is `e3nn.non_linearities.gated_block.GatedBlock` which applies a nonlinearity by gating each $L > 0$ channel with an added scalar channel.

In [ ]:
import e3nn.non_linearities as nl
from e3nn.non_linearities import rescaled_act

gated_block = nl.gated_block.GatedBlock(Rs_out, sp, rescaled_act.sigmoid)

dimensionalities = [2 * L + 1 for mult, L in Rs_out for _ in range(mult)]
gated_act = nl.GatedBlock(
    dimensionalities, rescaled_act.sigmoid, rescaled_act.sigmoid)